In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from chromedriver_py import binary_path
import time
import os
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
svc = webdriver.ChromeService(executable_path=binary_path)
driver = webdriver.Chrome(service=svc)
driver.maximize_window()

# Open the Python website
driver.get("https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin")

time.sleep(4)

username = driver.find_element(By.XPATH, "//input[@name='session_key']")
password = driver.find_element(By.XPATH, "//input[@name='session_password']")

username.send_keys(os.getenv("LINKEDIN_LOGIN"))
password.send_keys(os.getenv("LINKEDIN_PASSWORD"))

time.sleep(2)

submit = driver.find_element(By.XPATH, "//button[@type='submit']")
submit.click()
time.sleep(2)

driver.get(f"https://www.linkedin.com/jobs/search/?keywords={os.getenv('JOB_TITLE')}&location={os.getenv('JOB_LOCATION')}&sortBy=DD")

time.sleep(3)

links = []
print('Links are being collected now.')

try:
    for page in range(2, 3):
        time.sleep(2)
        jobs_list = driver.find_elements(By.CSS_SELECTOR, 'ul.scaffold-layout__list-container > li')

        for job in jobs_list:
            all_links = job.find_elements(By.CSS_SELECTOR, 'a')
            for a in all_links:
                if str(a.get_attribute('href')).startswith("https://www.linkedin.com/jobs/view") and a.get_attribute(
                        'href') not in links:
                    links.append(a.get_attribute('href'))
                else:
                    pass
            # scroll down for each job element
            driver.execute_script("arguments[0].scrollIntoView();", job)

        print(f'Collecting the links in the page: {page - 1}')
        # go to next page:
        driver.find_element(By.XPATH, f"//button[@aria-label='Page {page}']").click()
        time.sleep(1)
except:
    pass
print('Found ' + str(len(links)) + ' links for job offers')

# Create empty lists to store information
job_titles = []
company_names = []
company_locations = []
work_methods = []
post_dates = []
job_desc = []

i = 0
j = 1
# Visit each link one by one to scrape the information
print('Visiting the links and collecting information just started.')
for i in range(len(links)):
    try:
        driver.get(links[i])
        i = i + 1
        time.sleep(2)
        # Click See more.
        driver.find_element(By.CSS_SELECTOR, "button.artdeco-card__action").click()
        time.sleep(1)
    except:
        pass

    # Find the general information of the job offers
    contents = driver.find_elements(By.CSS_SELECTOR, 'div.p5 > div')
    for content in contents:
        try:
            job_titles.append(content.find_element(By.CSS_SELECTOR, "h1").text)
            company_names.append(driver.find_element(By.CSS_SELECTOR, 'div.job-details-jobs-unified-top-card__company-name a').text)
            company_locations.append(driver.find_element(By.CSS_SELECTOR, 'div.job-details-jobs-unified-top-card__tertiary-description span:nth-child(1)').text)

            # Find all elements containing the working methods information
            working_methods_elements = driver.find_elements(By.CSS_SELECTOR, 'span.ui-label.ui-label--accent-3.text-body-small')
            # Extract the text content from each element
            working_method = [element.text[:element.text.index("\n")] for element in working_methods_elements]
            work_methods.append(working_method)
            post_dates.append(driver.find_element(By.CSS_SELECTOR, 'div.job-details-jobs-unified-top-card__tertiary-description span:nth-child(3)').text)
            #work_times.append(content.find_element_by_class_name("jobs-unified-top-card__job-insight").text)
            print(len(job_titles), len(company_names), len(company_locations))
            j += 1
        except:
            pass

        # Scraping the job description
    job_description = driver.find_elements(By.CSS_SELECTOR, 'div.jobs-description__content')
    for description in job_description:
        job_text = description.find_element(By.CSS_SELECTOR, "div.jobs-box__html-content").text + '\n'
        job_desc.append(job_text)
        print(f'Scraping the Job Offer {j}')


# Creating the dataframe
datamap = {'job_title': job_titles, 'company_name': company_names, 'company_location': company_locations, 'job_types': work_methods, 'job_desc': job_desc}

# Verify DataFrame creation
df = pd.DataFrame(datamap)
print(df)  # Print the DataFrame head to check if it's created correctly

# Storing the data to csv file
df.to_csv('job_offers.csv', index=False)

# Close the browser window
driver.quit()

In [4]:
data = pd.read_csv("job_offers.csv")
data.head()

,job_title,company_name,company_location,job_types,job_desc
0,PHP Programmer,Apollo Interactive,"El Segundo, CA",[],About the job\nApollo Interactive is seeking a...
1,Senior BMC Firmware Engineer,Protingent,"Austin, TX","['$55/hr - $61/hr', 'On-site', 'Contract']",About the job\nJob Description\n\nPosition Tit...
2,Infrastructure Engineer,Mainz Brady Group,"Anacortes, WA","['Hybrid', 'Full-time']",About the job\nLead Engineer for GitHub Admini...
3,Senior Frontend Development Software Engineer,Ford Motor Company,"Dearborn, MI","['Hybrid', 'Full-time']",About the job\nJob Description\n\nWe are the m...
4,ADAS Embedded Software Performance Engineer,Ford Motor Company,"Dearborn, MI","['Hybrid', 'Full-time']",About the job\nResponsibilities\n\nWe are the ...
